# Bits and Bobbles
https://www.philipzucker.com/ground_kbo/


# Old notes String rewriting systems

[Semi-Thue systems](https://en.wikipedia.org/wiki/Semi-Thue_system)
[Word problem](https://en.wikipedia.org/wiki/Word_problem_(mathematics))
Monoid presentation

converting to term rewriting system fff ---> f(f(f(X)))

Term Rewriting using linux command line tools

The string search and manipulation tools are very powerful and efficient. They compile queries like grep into simple machines and such I think.

There's a big difference between ground and non ground terms. They appear subtly different when latexed, but the are way different beats
Ground terms equation proving can be done through the e graph efficiently.

Ground term rewriting seems to be identical to string rewriting. Just layout serially a traversal of the term.

The implicit prefix and suffix are the context of the term

```python

rules = [
  ("aa", "b"),
  ("b", "c")
]

def run_rules(s,rules):
  old_s = None
  while s != old_s:
    old_s = s
    for lhs,rhs in rules:
      s = s.replace(lhs,rhs)
  return s

print(run_rules("ababaaaaccaaa", rules))

def naive_completion(rules):  
  old_rules = None
  while rules != old_rules:
    old_rules = rules.copy()
    for (lhs,rhs) in old_rules:
        a = run_rules(lhs, rules)
        b = run_rules(rhs, rules)
        if a == b:
          break
        if a < b:
          rules.add((b,a))
        if a > b:
          rules.add((a,b))
  return rules

# an incomplete reduction routine?
# Triangular rewrite rules in some sense.
# Is this right at all? This is like a chunk of Huet's. I think just moving R -> E might be ok even if not one of listed rules. No, if I could do that ths move + simplfy would give me a more powerful R-simplify.
# I might be weakening my rules. That's not so bad imo.
def reduce_rules(E):  
  # worklist style
  R = set()
  # Sort E so smallest last probably. Most reduction power.
  E = sorted(E, key=lambda k: len(k[0]), reverse=True)
  while len(E) > 0:
    (a,b) = E.pop()
    a = run_rules(a, R) # simplify
    b = run_rules(b, R)
    print(a,b)
    if a == b: #delete
      continue
    if (len(b), b) > (len(a), a): # len then lex ordering
      R.add((b,a))
    if (len(a), a) > (len(b), b):
      R.add((a,b))
  return R



rules = {
  ("aaa", "a"),
  ("aaa", "c")
}

print(naive_completion(rules))


rules = [
  ("ffa", "a"),
  ("fa", "a")
]

print(run_rules("ffffffffffffa", rules))

print(reduce_rules(rules))

rules = [
  ("12+", "21+"), # an application of comm
  ("23+1+", "2+31+") # an application of assoc
]
# I am really going to want a notion of indirection or compression here.
# Intern strings

class RPN():
  def __init__(self, s):
    self.s = str(s)
  def __add__(self,b):
    return RPN(self.s + b.s + "+")
  def __repr__(self):
    return self.s

b0 = RPN(0)
b1 = RPN(1)
b2 = RPN(2)
b3 = RPN(3)


E = [
  (b1 + b0, b0 + b1),
  (b0 + b1, b1),
  (b1 + b2, b3),
  (b2, b1 + b1) 
]
E = [
  ("10+", "01+"),
  ("01+", "1"),
  ("12+", "3"),
  ("2", "11+"),
  ("00+", "0"),
  ("00+1+1+1+", "3"),
]

print(reduce_rules(E))
E = reduce_rules(E)
print(run_rules("00+1+0+1+0+2+",E))

E = [( str(i) + str(j) + "+" , str(i + j)) for i in range(4) for j in range(10) if i + j <= 9]
print(E)
print(reduce_rules(E))
print(run_rules("00+1+0+1+0+2+",E))

```

Ropes

We can of course compile a rule set to do better than this. In some sense every string represents a possibly infinite class of strings posible by running rules in reverse

String rewriting systems are a bit easier to think about and find good stock library functionality for.

string rewriting is unary term rewriting. A variable string pattern "aaaXbbbb" is a curious object from that perspective. While simple, it is a higher order pattern. `a(a(a(X(b(b(b(Y)))))))`. You can also finitize a bit. `foo(a)` can be made an atomic character. Or you can partially normalize a term rewriting system to string rewriting form

String orderings
Lexicographic comparison
Length
shortlex - first length, then lex
symbol counts

```python
def critical_pairs(a,b):
  assert len(a) > 0 and len(b) > 0
  if len(b) <= len(a):
    a,b = b,a # a is shorter
  cps = []
  if b.find(a) != -1: # b contains a
   cps.append(b)
  for n in range(1,len(a)): # check if overlapping concat is possible
    if b[-n:] == a[:n]:
      cps.append(b + a[n:])
    if b[:n] == a[-n:]:
      cps.append(a + b[n:])
  return cps

print(critical_pairs("aba", "ac"))
print(critical_pairs("aba", "ca"))
print(critical_pairs("abab", "ba"))
  
'''
def reduce_rules(rules): # a very simple reduction, reducing rhs, and removing duplicate rules
  rules1 = set()
  for (lhs,rhs) in rules:
    rhs = run_rules(rhs,rules)
    rules1.add((lhs,rhs))
  return rules1
'''

  #run_rules
  #reduce_rules
```

Building a suffix tree might be a good way to find critical pairs.

Lempel Ziv / Lzw is the analog of hash consing? Some kind of string compression is. That's fun.

<http://haskellformaths.blogspot.com/2010/05/string-rewriting-and-knuth-bendix.html>

It seems like named pattern string rewriting and variable term rewriting might be close enough

You could imagine

((x + 0) + 0 + 0) laid out as + + + x 0 0 0. and the found ground rewite rule + x 0 -> x being applied iteratively.

Labelling shifts:
f(g(a), b) ->   f +0 g +1 a -1 b -0

the pattern
f(?x)  -> ?x
 becoming
 f +0 <x> -0 -> <x>
 f +1 <x> -1 -> <x>
 f +2 <x> -2 -> <x>
and so on to some upper limit
We could occasionally renormalize maybe if there are no +n -n pairs remaining.
then shuffle all the above ones down.
Ok but what about something that increases the depth
x -> f(x)
... hmmm.

And if we number from the bottom?

f +2 <X> -2 -> <X>

and
<X> -> ??? Well a raw pattern is pretty shitty
f(x) -> f(f(x)) becomes
f +n <X> -n -> f +(n+1) f +n <x>
Yeah. Numbering from the bottom is better. We don't have to

f(stuff,stuff)
f +n <X> -n +n <Y> -n

even without enumerating
plus +<n1> <X> -<n1> +<n2> <Y> -<n2>

Oooh. We have to enumerate every combo of possible subterm depths.

Hmm. This adjust levels

<http://matt.might.net/articles/sculpting-text/>

A unique terminator for the subexpression is the point.
f +2 (^ -2)* -2

could have a counter per symbol. per symbol depth.
f1 ( yadayada) \f1

fa1 <X> fb1 <Y> fc1

huh. What about the CPP? Won't that basically work?

This is horribly inefficent because it'll expand out huge terms.
And big backtracking jumps. Or rather big seeks while it tries to find the next spot to go to. The next argument of f.

For ground term rewriting it seems actually reasonable and faster than having indirections. We can't have sharing though. That is a bummer.Maybe with zip.
Unless our goal is simplifcation.

using the rule

We could try to use the e-matching strategy where we iteratively instantiate fixed ground rewrite rules into the sed file itself?

Instead of using parenthesis, one could use numbered enter level exit level. And then bound the number of levels.
Each term rewriting becomes a string rewriting (with named regex holes ) replicated by the number of supported levels.

using sed on itself we might be able to implement knuth bendix

One could compile into a human unreadable huffman encoded like thing

<https://superuser.com/questions/1366825/how-to-recursively-replace-characters-with-sed> looping in sed
You can gnu parallel into chunks
grep is much faster. If terms are rare, maybe find using grep first?

Suffix trees can store every subterm of ground term with efficient query.

<https://spencermortensen.com/articles/serialize-a-binary-tree-in-minimal-space/> an interesting perspective on tree serialization
catalan numbers. We know size of tree if we give elements.
<https://www.educative.io/edpresso/what-is-morris-traversal> woah. Bizarre. It mutates the true as it goes down it and store
Kind of reminscent of dancing links in a way

f 20 90 190 yada yada yada could desribe links to the approprate spots.
This would be the analog of flatterms.

There is something like encoding lambda terms with de bruijn. vs unique signifiers.
If we could encode the unique signifiers in a way such that they never collide.

There is something to that we're kind of converting to rpn.
<https://github.com/GillesArcas/numsed>
<https://github.com/shinh/sedlisp>